In [2]:
# trying to install on 3.11 fails trying to build wheel files; see https://github.com/yaml/pyyaml/issues/724
!rye remove protobuf
!rye add pyyaml==5.3 mlrun~=1.6.4  scikit-learn~=1.4.0  numpy~=1.26 protobuf==3.20.1


Removed protobuf==3.14
Reusing already existing virtualenv
Generating production lockfile: /Users/bcorfman/dev/mlrun-trial/requirements.lock
Generating dev lockfile: /Users/bcorfman/dev/mlrun-trial/requirements-dev.lock
Installing dependencies
Building file:///Users/bcorfman/dev/mlrun-trial                
   Built file:///Users/bcorfman/dev/mlrun-trial   
   Built file:///Users/bcorfman/dev/mlrun-trial   
   Built file:///Users/bcorfman/dev/mlrun-trialv/m
   Built file:///Users/bcorfman/dev/mlrun-trial                                 Built 1 editable in 526ms
Uninstalled 1 package in 1ms
Installed 1 package in 1ms==0.1.0 (from file:///Users/bcorfman/dev/m
 - mlrun-trial==0.1.0 (from file:///Users/bcorfman/dev/mlrun-trial)
 + mlrun-trial==0.1.0 (from file:///Users/bcorfman/dev/mlrun-trial)
Done!
Added pyyaml==5.3 as regular dependency
Added mlrun~=1.6.4 as regular dependency
Added scikit-learn~=1.4.0 as regular dependency
Added numpy~=1.26 as regular dependency
Added protobuf==3.20.1 a

In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
import mlrun

ImportError: cannot import name '_message' from 'google.protobuf.pyext' (/Users/bcorfman/dev/mlrun-trial/.venv/lib/python3.9/site-packages/google/protobuf/pyext/__init__.py)

In [ ]:
project = mlrun.get_or_create_project("quick-tutorial", "./", user_project=True)

In [ ]:
%%writefile data-prep.py

import pandas as pd
from sklearn.datasets import load_breast_cancer

import mlrun


@mlrun.handler(outputs=["dataset", "label_column"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["label"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset, "label"

In [ ]:
data_gen_fn = project.set_function(
    "data-prep.py",
    name="data-prep",
    kind="job",
    image="mlrun/mlrun",
    handler="breast_cancer_generator",
)
project.save()  # save the project with the latest config